# Hybrid reliability methods

In this example, we demonstrate the usage of the so-called hybrid reliability methods:
* FDIR (`form_then_directional_sampling`) - Reliability calculations are first performed using FORM. If FORM does not converge, the calculations are completed using Directional Sampling.
* DSFI (`directional_sampling_then_form`) - Reliability calculations are initially performed with Directional Sampling, after which FORM is used to re-calculated the design point.

### Define model

First, let's import the necessary package:

In [ ]:
from probabilistic_library import ReliabilityMethod, ReliabilityProject, DistributionType

Next, we define a simple limit state function: 

$Z = 1.9 - (a+b)$

In [2]:
from utils.models import linear_a_b

### Define reliability project

We define the reliability project and the random variables $a$ and $b$:

In [10]:
project = ReliabilityProject()
project.model = linear_a_b

project.variables["a"].distribution = DistributionType.uniform
project.variables["a"].minimum = -1
project.variables["a"].maximum = 1

project.variables["b"].distribution = DistributionType.uniform
project.variables["b"].minimum = -1
project.variables["b"].maximum = 1

### Run calculations with FORM

We begin by running `form` calculations and consider two cases: one with convergence and one without.

### With convergence

In [ ]:
project.settings.reliability_method = ReliabilityMethod.form

project.settings.relaxation_factor = 0.15
project.settings.maximum_iterations = 50
project.settings.variation_coefficient = 0.02

project.run()

from utils.printing import print_hybrid
print_hybrid(project.design_point, project)

Beta = 2.7729135494928867
Probability of failure = 0.002777843889340099
a: alpha = -0.7071067811865476, x = 0.9500913368925314
b: alpha = -0.7071067811865476, x = 0.9500913368925314
Converged (convergence = 0.0083829374389322 < 0.02)
Model runs = 126


#### Without convergence

By reducing the maximum number of iterations, we create a case where `form` does not converge:

In [13]:
project.settings.reliability_method = ReliabilityMethod.form

project.settings.relaxation_factor = 0.15
project.settings.maximum_iterations = 10
project.settings.variation_coefficient = 0.02

project.run()

print_hybrid(project.design_point, project)

Beta = 2.3484209552730166
Probability of failure = 0.009426600189324523
a: alpha = -0.7071067811865476, x = 0.9032030522941867
b: alpha = -0.7071067811865476, x = 0.9032030522941867
Not converged (convergence = 1.2240363751044934 > 0.02)
Model runs = 30


### Calculations with Directional Sampling

We also run the reliability calculations with `directional_sampling`:

In [6]:
project.settings.reliability_method = ReliabilityMethod.directional_sampling

project.settings.variation_coefficient = 0.02

project.settings.minimum_directions = 10000
project.settings.maximum_directions = 50000

project.run()

print_hybrid(project.design_point, project)

Beta = 3.0177974723640384
Probability of failure = 0.0012730950337967344
a: alpha = -0.7064707502224403, x = 0.9669919732731047
b: alpha = -0.7077422405651244, x = 0.9673061355044628
Converged (convergence = 0.01999856184538342 < 0.02)
Model runs = 145496


### Run calculations with fdir

We now perform calculations using `form_then_directional_sampling`.

If `form` converges, the results should match the `form` results.

In [7]:
project.settings.reliability_method = ReliabilityMethod.form_then_directional_sampling

project.settings.relaxation_factor = 0.15
project.settings.maximum_iterations = 50
project.settings.variation_coefficient = 0.02

project.settings.minimum_directions = 10000
project.settings.maximum_directions = 20000

project.run()

print_hybrid(project.design_point, project)

Beta = 2.7729135494928867
Probability of failure = 0.002777843889340099
a: alpha = -0.7071067811865476, x = 0.9500913368925314
b: alpha = -0.7071067811865476, x = 0.9500913368925314
Converged (convergence = 0.0083829374389322 < 0.02)
Model runs = 126


When `form` does not converge, the results should match those of `directional_sampling`:

In [8]:
project.settings.reliability_method = ReliabilityMethod.form_then_directional_sampling

project.settings.relaxation_factor = 0.15
project.settings.maximum_iterations = 10
project.settings.variation_coefficient = 0.02

project.settings.minimum_directions = 10000
project.settings.maximum_directions = 50000

project.run()

print_hybrid(project.design_point, project)

Beta = 3.0177974723640384
Probability of failure = 0.0012730950337967344
a: alpha = -0.7064707502224403, x = 0.9669919732731047
b: alpha = -0.7077422405651244, x = 0.9673061355044628
Converged (convergence = 0.01999856184538342 < 0.02)
Model runs = 145496


### Calculations with dsfi

We now perform reliability calculations using `directional_sampling_then_form`. The resulting reliability index (beta) must match the reliability index calculated with `directional_sampling`. However, the design point is re-calculated using `form` and, therefore, differs from the one obtained with `directional_sampling`.

In [9]:
project.settings.reliability_method = ReliabilityMethod.directional_sampling_then_form

project.settings.relaxation_factor = 0.15
project.settings.maximum_iterations = 50
project.settings.variation_coefficient = 0.02

project.settings.minimum_directions = 10000
project.settings.maximum_directions = 50000

project.run()

print_hybrid(project.design_point, project)

Beta = 3.0177974723640384
Probability of failure = 0.0012730950337967344
a: alpha = -0.7071067811866524, x = 0.9498944711126716
b: alpha = -0.7071067811864427, x = 0.9498944711126036
Direction sampling step:
Converged (convergence = 0.01999856184538342 < 0.02)
Model runs = 145496
FORM step:
Converged (convergence = 0.009011403674565818 < 0.02)
Model runs = 54
